### train 데이터셋에 대한 augmentation 적용
- Augmentation 적용 내용  

            - 90도 랜덤 회전  
            - 랜덤 회전
            - 90도 랜덤 회전 + 수평 플립  
            - 랜덤 회전 + 수평 플립  
            - 90도 랜덤 회전 + 수직 플립
            - 랜덤 회전 + 수직 플립            
            - crop out 적용   

In [16]:
def yolo_to_coco(bbox, image_width, image_height):
    """
    Convert YOLO format bounding box to COCO format.
    
    Args:
        bbox (list float): YOLO format bounding box [x_center, y_center, width, height].
        image_width (int): Width of the image.
        image_height (int): Height of the image.
        
    Returns:
        list int: COCO format bounding box [x_min, y_min, box_width, box_height].
    """
    x_center, y_center, width, height = bbox
    x_min = min(image_width-1, max(0, round((x_center - width / 2) * image_width)))
    y_min = min(image_height-1, max(0, round((y_center - height / 2) * image_height)))
    box_width = max(1, min(image_width-1-x_min, round((width) * image_width)))
    box_height = max(1, min(image_height-1-y_min, round((height) * image_height)))
    
    return [x_min, y_min, box_width, box_height]

def coco_to_yolo(bbox, image_width, image_height):
    """
    Convert COCO format bounding box to YOLO format.
    
    Args:
        bbox (list int): COCO format bounding box [x_min, y_min, box_width, box_height].
        image_width (int): Width of the image.
        image_height (int): Height of the image.
        
    Returns:
        list float: YOLO format bounding box [x_center, y_center, width, height].
    """
    x_min, y_min, box_width, box_height = bbox
    x_center = min(1., (x_min + box_width / 2.) / image_width)
    y_center = min(1., (y_min + box_height / 2.) / image_height)
    width = min(1., (box_width) / image_width)
    height = min(1., (box_height) / image_height)
    
    return [x_center, y_center, width, height]

def yolo_to_pascal_voc(bbox, image_width, image_height):
    """
    Convert YOLO format bounding box to Pascal VOC format.
    
    Args:
        bbox (list): YOLO format bounding box [x_center, y_center, width, height].
        image_width (int): Width of the image.
        image_height (int): Height of the image.
        
    Returns:
        list: Pascal VOC format bounding box [x_min, y_min, x_max, y_max].
    """
    x_center, y_center, width, height = bbox
    x_min = max(0, min(image_width-2, round((x_center - width / 2) * image_width)))
    y_min = max(0, min(image_height-2, round((y_center - height / 2) * image_height)))
    x_max = min(image_width-1, max(x_min+1, round((x_center + width / 2) * image_width)))
    y_max = min(image_height-1, max(y_min+1, round((y_center + height / 2) * image_height)))
    
    return [x_min, y_min, x_max, y_max]

def pascal_voc_to_yolo(bbox, image_width, image_height):
    """
    Convert Pascal VOC format bounding box to YOLO format.
    
    Args:
        bbox (list): Pascal VOC format bounding box [x_min, y_min, x_max, y_max].
        image_width (int): Width of the image.
        image_height (int): Height of the image.
        
    Returns:
        list: YOLO format bounding box [x_center, y_center, width, height].
    """
    x_min, y_min, x_max, y_max = bbox
    x_center = (x_min + x_max) / 2. / image_width
    y_center = (y_min + y_max) / 2. / image_height
    width = (x_max - x_min) / image_width
    height = (y_max - y_min) / image_height
    
    return [x_center, y_center, width, height]


In [2]:
# copy all the images in 'test/images' folder and save rotated version of those in 'test_aug/images' folder
# test/images는 원본 파일로 수정되거나 삭제되지 않도록 주의해야 함. 모든 변경 생성되는 파일은 'test_aug' 폴더에 저장되도록 해야 함.
import cv2
import os
import random
import glob
import shutil
import albumentations as A

category_ids = []
category_id_to_name = {0: 'human'}

SOURCE_PATH = 'C:/Users/User2/source/pcounter/data/people/train'
SOURCE_DATA_PATH = f'{SOURCE_PATH}'+'/images'
SOURCE_LABEL_PATH = f'{SOURCE_PATH}'+'/labels'

TARGET_PATH = 'C:/Users/User2/source/pcounter/data/people/train_aug'
TARGET_DATA_PATH = f'{TARGET_PATH}'+'/images'
TARGET_LABEL_PATH = f'{TARGET_PATH}'+'/labels'

image_files = os.listdir(SOURCE_DATA_PATH)


In [3]:
# ==> 90도 랜덤 회전  
# - 랜덤 회전 
# - 90도 랜덤 회전 + 수평 플립  
# - 랜덤 회전 + 수평 플립  
# - 90도 랜덤 회전 + 수직 플립
# - 랜덤 회전 + 수직 플립  
# - random crop 적용 
augment_type = 'rot90_'
for filename in image_files:    
    pat, ext = os.path.splitext(filename)
    src_image_file = f'{SOURCE_DATA_PATH}/'+filename
    src_label_file = f'{SOURCE_LABEL_PATH}/'+pat+'.txt'        
    print(src_image_file)
    print(src_label_file)

    dst_image_file = f'{TARGET_DATA_PATH}/'+augment_type+filename     
    dst_label_file = f'{TARGET_LABEL_PATH}/'+augment_type+pat+'.txt'  
    print(dst_image_file)
    print(dst_label_file)

    bboxes = []
    category_ids = []

    image = cv2.imread(src_image_file, cv2.IMREAD_COLOR)    

    if os.path.isfile(src_label_file):
        f = open(src_label_file, 'r')
        for label in f.readlines():
            x = label.split()
            class_id = x.pop(0)

            x = yolo_to_pascal_voc([float(x[0]), float(x[1]), float(x[2]), float(x[3])], image.shape[1], image.shape[0])
            x.append(str(class_id))
            bboxes.append(x)
        f.close()

    print(len(bboxes), len(category_ids))
    print(bboxes)
    print(category_ids)

    random.seed()    
    transform = A.Compose(
        [
        #  A.Rotate(border_mode=cv2.BORDER_CONSTANT, value=[0,0,0], p=1)
         A.RandomRotate90(p=1)
        ],
        bbox_params=A.BboxParams(format='pascal_voc'), #현재 albumentations의 버그 때문에 pascal_voc 형태로 변형하고 마지막에 다시 yolo 형태로 변형함
    ) 
    transformed = transform(image=image, bboxes=bboxes)
    transformed_image = transformed['image']
    transformed_bboxes = transformed['bboxes']

    cv2.imwrite(dst_image_file, transformed_image)
    print(transformed_bboxes)

    if len(bboxes) > 0:
        f = open(dst_label_file, 'w')
        for label in transformed_bboxes:
            class_id = label[4]
            x = [label[0], label[1], label[2], label[3]]
            x = pascal_voc_to_yolo(x, transformed_image.shape[1], transformed_image.shape[0])
            f.write(f'{int(class_id)} {x[0]:.6f} {x[1]:.6f} {x[2]:.6f} {x[3]:.6f}\n')
        f.close()

C:/Users/User2/source/pcounter/data/people/train/images/p_train0000.png
C:/Users/User2/source/pcounter/data/people/train/labels/p_train0000.txt
C:/Users/User2/source/pcounter/data/people/train_aug/images/rot90_p_train0000.png
C:/Users/User2/source/pcounter/data/people/train_aug/labels/rot90_p_train0000.txt
3 0
[[307, 65, 373, 154, '0'], [440, 38, 488, 104, '0'], [275, 339, 389, 358, '0']]
[]
[(267.0, 206.0, 333.0, 295.0, '0'), (152.00000000000003, 256.0, 200.0, 322.0, '0'), (251.0, 1.999999999999993, 365.0, 21.000000000000007, '0')]
C:/Users/User2/source/pcounter/data/people/train/images/p_train0001.png
C:/Users/User2/source/pcounter/data/people/train/labels/p_train0001.txt
C:/Users/User2/source/pcounter/data/people/train_aug/images/rot90_p_train0001.png
C:/Users/User2/source/pcounter/data/people/train_aug/labels/rot90_p_train0001.txt
5 0
[[1, 23, 88, 92, '0'], [177, 84, 277, 158, '0'], [384, 83, 488, 158, '0'], [401, 1, 506, 106, '0'], [283, 335, 399, 359, '0']]
[]
[(552.0, 268.0, 639

In [4]:
# - 90도 랜덤 회전
# ==> 랜덤 회전 
# - 90도 랜덤 회전 + 수평 플립  
# - 랜덤 회전 + 수평 플립  
# - 90도 랜덤 회전 + 수직 플립
# - 랜덤 회전 + 수직 플립     
# - random crop 적용 
augment_type = 'rot_'
for filename in image_files:    
    pat, ext = os.path.splitext(filename)
    src_image_file = f'{SOURCE_DATA_PATH}/'+filename
    src_label_file = f'{SOURCE_LABEL_PATH}/'+pat+'.txt'        
    print(src_image_file)
    print(src_label_file)

    dst_image_file = f'{TARGET_DATA_PATH}/'+augment_type+filename     
    dst_label_file = f'{TARGET_LABEL_PATH}/'+augment_type+pat+'.txt'  
    print(dst_image_file)
    print(dst_label_file)

    bboxes = []
    category_ids = []

    image = cv2.imread(src_image_file, cv2.IMREAD_COLOR)    

    if os.path.isfile(src_label_file):
        f = open(src_label_file, 'r')
        for label in f.readlines():
            x = label.split()
            class_id = x.pop(0)

            x = yolo_to_pascal_voc([float(x[0]), float(x[1]), float(x[2]), float(x[3])], image.shape[1], image.shape[0])
            x.append(str(class_id))
            bboxes.append(x)
        f.close()

    print(len(bboxes), len(category_ids))
    print(bboxes)
    print(category_ids)

    random.seed()    
    transform = A.Compose(
        [
            A.Rotate(border_mode=cv2.BORDER_CONSTANT, value=[0,0,0], p=1)
        ],
        bbox_params=A.BboxParams(format='pascal_voc'), #현재 albumentations의 버그 때문에 pascal_voc 형태로 변형하고 마지막에 다시 yolo 형태로 변형함
    ) 
    transformed = transform(image=image, bboxes=bboxes)
    transformed_image = transformed['image']
    transformed_bboxes = transformed['bboxes']

    cv2.imwrite(dst_image_file, transformed_image)
    print(transformed_bboxes)

    if len(bboxes) > 0:
        f = open(dst_label_file, 'w')
        for label in transformed_bboxes:
            class_id = label[4]
            x = [label[0], label[1], label[2], label[3]]
            x = pascal_voc_to_yolo(x, transformed_image.shape[1], transformed_image.shape[0])
            f.write(f'{int(class_id)} {x[0]:.6f} {x[1]:.6f} {x[2]:.6f} {x[3]:.6f}\n')
        f.close()

C:/Users/User2/source/pcounter/data/people/train/images/p_train0000.png
C:/Users/User2/source/pcounter/data/people/train/labels/p_train0000.txt
C:/Users/User2/source/pcounter/data/people/train_aug/images/rot_p_train0000.png
C:/Users/User2/source/pcounter/data/people/train_aug/labels/rot_p_train0000.txt
3 0
[[307, 65, 373, 154, '0'], [440, 38, 488, 104, '0'], [275, 339, 389, 358, '0']]
[]
[(328.3366354526333, 87.30681199499077, 437.38385285615766, 197.35821060805785, '0'), (459.48738805776657, 158.7315990709782, 539.6661967002461, 239.69628866005633, '0'), (165.29374391102743, 265.0250744798378, 261.3679703774047, 356.95159595027775, '0')]
C:/Users/User2/source/pcounter/data/people/train/images/p_train0001.png
C:/Users/User2/source/pcounter/data/people/train/labels/p_train0001.txt
C:/Users/User2/source/pcounter/data/people/train_aug/images/rot_p_train0001.png
C:/Users/User2/source/pcounter/data/people/train_aug/labels/rot_p_train0001.txt
5 0
[[1, 23, 88, 92, '0'], [177, 84, 277, 158, '0

In [5]:
# - 90도 랜덤 회전
# - 랜덤 회전 
# ==> 90도 랜덤 회전 + 수평 플립
# - 랜덤 회전 + 수평 플립  
# - 90도 랜덤 회전 + 수직 플립
# - 랜덤 회전 + 수직 플립     
# - random crop 적용

augment_type = 'rot90HF_'
for filename in image_files:    
    pat, ext = os.path.splitext(filename)
    src_image_file = f'{SOURCE_DATA_PATH}/'+filename
    src_label_file = f'{SOURCE_LABEL_PATH}/'+pat+'.txt'        
    print(src_image_file)
    print(src_label_file)

    dst_image_file = f'{TARGET_DATA_PATH}/'+augment_type+filename     
    dst_label_file = f'{TARGET_LABEL_PATH}/'+augment_type+pat+'.txt'  
    print(dst_image_file)
    print(dst_label_file)

    bboxes = []
    category_ids = []

    image = cv2.imread(src_image_file, cv2.IMREAD_COLOR)    

    if os.path.isfile(src_label_file):
        f = open(src_label_file, 'r')
        for label in f.readlines():
            x = label.split()
            class_id = x.pop(0)

            x = yolo_to_pascal_voc([float(x[0]), float(x[1]), float(x[2]), float(x[3])], image.shape[1], image.shape[0])
            x.append(str(class_id))
            bboxes.append(x)
        f.close()

    print(len(bboxes), len(category_ids))
    print(bboxes)
    print(category_ids)

    random.seed()    
    transform = A.Compose(
        [
            # A.Rotate(border_mode=cv2.BORDER_CONSTANT, value=[0,0,0], p=1)
            A.RandomRotate90(p=1),
            A.HorizontalFlip(p=1)
        ],
        bbox_params=A.BboxParams(format='pascal_voc'), #현재 albumentations의 버그 때문에 pascal_voc 형태로 변형하고 마지막에 다시 yolo 형태로 변형함
    ) 
    transformed = transform(image=image, bboxes=bboxes)
    transformed_image = transformed['image']
    transformed_bboxes = transformed['bboxes']

    cv2.imwrite(dst_image_file, transformed_image)
    print(transformed_bboxes)

    if len(bboxes) > 0:
        f = open(dst_label_file, 'w')
        for label in transformed_bboxes:
            class_id = label[4]
            x = [label[0], label[1], label[2], label[3]]
            x = pascal_voc_to_yolo(x, transformed_image.shape[1], transformed_image.shape[0])
            f.write(f'{int(class_id)} {x[0]:.6f} {x[1]:.6f} {x[2]:.6f} {x[3]:.6f}\n')
        f.close()

C:/Users/User2/source/pcounter/data/people/train/images/p_train0000.png
C:/Users/User2/source/pcounter/data/people/train/labels/p_train0000.txt
C:/Users/User2/source/pcounter/data/people/train_aug/images/rot90HF_p_train0000.png
C:/Users/User2/source/pcounter/data/people/train_aug/labels/rot90HF_p_train0000.txt
3 0
[[307, 65, 373, 154, '0'], [440, 38, 488, 104, '0'], [275, 339, 389, 358, '0']]
[]
[(267.0, 65.0, 333.0, 154.0, '0'), (152.00000000000003, 38.0, 200.0, 103.99999999999999, '0'), (251.0, 339.0, 365.0, 358.0, '0')]
C:/Users/User2/source/pcounter/data/people/train/images/p_train0001.png
C:/Users/User2/source/pcounter/data/people/train/labels/p_train0001.txt
C:/Users/User2/source/pcounter/data/people/train_aug/images/rot90HF_p_train0001.png
C:/Users/User2/source/pcounter/data/people/train_aug/labels/rot90HF_p_train0001.txt
5 0
[[1, 23, 88, 92, '0'], [177, 84, 277, 158, '0'], [384, 83, 488, 158, '0'], [401, 1, 506, 106, '0'], [283, 335, 399, 359, '0']]
[]
[(23.0, 1.0, 92.0, 88.0, 

In [8]:
bboxes = [[0, 0.592187, 0.404167, 0.150000, 0.180556],
[0, 0.783594, 0.597222, 0.132812, 0.133333],
[0, 0.332813, 0.862500, 0.150000, 0.252778]]

for bbox in bboxes:
    x = bbox
    class_id = x.pop(0)

    x = yolo_to_coco([float(x[0]), float(x[1]), float(x[2]), float(x[3])], 640, 360)
    x.append(str(class_id))
    print(x)

[331, 113, 427, 178, '0']
[459, 191, 544, 239, '0']
[165, 265, 261, 356, '0']


[331, 113, 96, 65, '0']
[459, 191, 85, 48, '0']
[165, 265, 96, 91, '0']


[165, 265, 261, 356, '0']

In [17]:
# - 90도 랜덤 회전
# - 랜덤 회전 
# - 90도 랜덤 회전 + 수평 플립  
# ==> 랜덤 회전 + 수평 플립  
# - 90도 랜덤 회전 + 수직 플립
# - 랜덤 회전 + 수직 플립     
# - random crop 적용

# 추가 수정 사항 : 수평 수직 플립 적용시 rot를 하게 됨에 따른 재라벨링 작업을 없애기 위함.
#                 rot + 재라벨링 결과를 가지고 수평 수직 플립만 추가하도록 하여 재라벨링 작업 제거
# 수정이유 : rot하면 rot변형 라벨링 과정에서 bbox가 커지기 때문에 다시 수동으로 재라벨링해야 함
# 수정방향 : rot한 + 재라벨링 한 결과에 H/V flip만 하여 재라벨링 작업 제거 하려고 함.
# 준비사항 : train_aug폴더 아래에 train_rot 폴더를 만들고 이전에 rot한 뒤 라벨링한 images와 labels를 동일한 폴더 구조로 카피한다.
#  

SOURCE_PATH = 'C:/Users/User2/source/pcounter/data/people/train_aug/train_rot'
SOURCE_DATA_PATH = f'{SOURCE_PATH}'+'/images'
SOURCE_LABEL_PATH = f'{SOURCE_PATH}'+'/labels'

TARGET_PATH = 'C:/Users/User2/source/pcounter/data/people/train_aug'
TARGET_DATA_PATH = f'{TARGET_PATH}'+'/images'
TARGET_LABEL_PATH = f'{TARGET_PATH}'+'/labels'

image_files = os.listdir(SOURCE_DATA_PATH)

augment_type = 'rotHF_'
for filename in image_files:    
    pat, ext = os.path.splitext(filename)
    src_image_file = f'{SOURCE_DATA_PATH}/'+filename
    src_label_file = f'{SOURCE_LABEL_PATH}/'+pat+'.txt'        
    print(src_image_file)
    print(src_label_file)

    dst_image_file = f'{TARGET_DATA_PATH}/'+augment_type+filename     
    dst_label_file = f'{TARGET_LABEL_PATH}/'+augment_type+pat+'.txt'  
    print(dst_image_file)
    print(dst_label_file)

    bboxes = []
    category_ids = []

    image = cv2.imread(src_image_file, cv2.IMREAD_COLOR)    
    print(f'image size : {image.shape[1]}, {image.shape[0]}')

    if os.path.isfile(src_label_file):
        f = open(src_label_file, 'r')
        ##### yolo -> pascal_voc 변환 + make bboxes
        # for label in f.readlines():
        #     x = label.split()
        #     class_id = x.pop(0)

        #     x = yolo_to_pascal_voc([float(x[0]), float(x[1]), float(x[2]), float(x[3])], image.shape[1], image.shape[0])
        #     x.append(str(class_id))
        #     bboxes.append(x)

        ##### yolo make bboxes
        # for label in f.readlines():
        #     x = label.split()
        #     class_id = x.pop(0)
        #     bbox = [float(x[0]), float(x[1]), float(x[2]), float(x[3]), str(class_id)]
        #     bboxes.append(bbox)
        # f.close()
        
        ##### yolo -> coco 변환 + make bboxes
        for label in f.readlines():
            x = label.split()
            class_id = x.pop(0)

            x = yolo_to_coco([float(x[0]), float(x[1]), float(x[2]), float(x[3])], image.shape[1], image.shape[0])
            x.append(str(class_id))
            bboxes.append(x)

    print(len(bboxes), len(category_ids))
    print(bboxes)
    print(category_ids)

    random.seed()    
    transform = A.Compose(
        [
            # A.Rotate(border_mode=cv2.BORDER_CONSTANT, value=[0,0,0], p=1),
            # A.RandomRotate90(p=1),
            A.HorizontalFlip(p=1)
        ],
        # bbox_params=A.BboxParams(format='pascal_voc'), #현재 albumentations의 버그(HorizontalFlip()) 때문에 pascal_voc 형태에서 오류 발생함.
        # bbox_params=A.BboxParams(format='yolo'), #현재 albumentations의 버그 때문에 yolo 형태로 변형함 => yolo 형태도 오류 발생함
        bbox_params=A.BboxParams(format='coco'), #현재 albumentations의 버그 때문에 coco 형태로 변형함 => yolo_to_coco(), coco_to_yolo() 함수에서 오류가 있어 수정함

    ) 
    transformed = transform(image=image, bboxes=bboxes)
    transformed_image = transformed['image']
    transformed_bboxes = transformed['bboxes']

    cv2.imwrite(dst_image_file, transformed_image)
    print(transformed_bboxes)

    if len(bboxes) > 0:
        f = open(dst_label_file, 'w')
        for label in transformed_bboxes:
            class_id = label[4]
            x = [label[0], label[1], label[2], label[3]]
            # x = pascal_voc_to_yolo(x, transformed_image.shape[1], transformed_image.shape[0])
            x = coco_to_yolo(x, transformed_image.shape[1], transformed_image.shape[0])
            f.write(f'{int(class_id)} {x[0]:.6f} {x[1]:.6f} {x[2]:.6f} {x[3]:.6f}\n')
        f.close()

C:/Users/User2/source/pcounter/data/people/train_aug/train_rot/images/rot_p_train0000.png
C:/Users/User2/source/pcounter/data/people/train_aug/train_rot/labels/rot_p_train0000.txt
C:/Users/User2/source/pcounter/data/people/train_aug/images/rotHF_rot_p_train0000.png
C:/Users/User2/source/pcounter/data/people/train_aug/labels/rotHF_rot_p_train0000.txt
image size : 640, 360
3 0
[[331, 113, 96, 65, '0'], [459, 191, 85, 48, '0'], [165, 265, 96, 91, '0']]
[]
[(212.99999999999997, 113.0, 96.00000000000003, 65.0, '0'), (96.00000000000001, 191.0, 84.99999999999999, 48.0, '0'), (379.0, 265.0, 96.0, 91.0, '0')]
C:/Users/User2/source/pcounter/data/people/train_aug/train_rot/images/rot_p_train0001.png
C:/Users/User2/source/pcounter/data/people/train_aug/train_rot/labels/rot_p_train0001.txt
C:/Users/User2/source/pcounter/data/people/train_aug/images/rotHF_rot_p_train0001.png
C:/Users/User2/source/pcounter/data/people/train_aug/labels/rotHF_rot_p_train0001.txt
image size : 640, 360
5 0
[[1, 246, 80, 

In [7]:
# - 90도 랜덤 회전
# - 랜덤 회전 
# - 90도 랜덤 회전 + 수평 플립  
# - 랜덤 회전 + 수평 플립  
# ==> 90도 랜덤 회전 + 수직 플립
# - 랜덤 회전 + 수직 플립     
# - random crop 적용
augment_type = 'rot90VF_'
for filename in image_files:    
    pat, ext = os.path.splitext(filename)
    src_image_file = f'{SOURCE_DATA_PATH}/'+filename
    src_label_file = f'{SOURCE_LABEL_PATH}/'+pat+'.txt'        
    print(src_image_file)
    print(src_label_file)

    dst_image_file = f'{TARGET_DATA_PATH}/'+augment_type+filename     
    dst_label_file = f'{TARGET_LABEL_PATH}/'+augment_type+pat+'.txt'  
    print(dst_image_file)
    print(dst_label_file)

    bboxes = []
    category_ids = []

    image = cv2.imread(src_image_file, cv2.IMREAD_COLOR)    

    if os.path.isfile(src_label_file):
        f = open(src_label_file, 'r')
        for label in f.readlines():
            x = label.split()
            class_id = x.pop(0)

            x = yolo_to_pascal_voc([float(x[0]), float(x[1]), float(x[2]), float(x[3])], image.shape[1], image.shape[0])
            x.append(str(class_id))
            bboxes.append(x)
        f.close()

    print(len(bboxes), len(category_ids))
    print(bboxes)
    print(category_ids)

    random.seed()    
    transform = A.Compose(
        [
            # A.Rotate(border_mode=cv2.BORDER_CONSTANT, value=[0,0,0], p=1),
            A.RandomRotate90(p=1),
            # A.HorizontalFlip(p=1),
            A.VerticalFlip(p=1)
        ],
        bbox_params=A.BboxParams(format='pascal_voc'), #현재 albumentations의 버그 때문에 pascal_voc 형태로 변형하고 마지막에 다시 yolo 형태로 변형함
    ) 
    transformed = transform(image=image, bboxes=bboxes)
    transformed_image = transformed['image']
    transformed_bboxes = transformed['bboxes']

    cv2.imwrite(dst_image_file, transformed_image)
    print(transformed_bboxes)

    if len(bboxes) > 0:
        f = open(dst_label_file, 'w')
        for label in transformed_bboxes:
            class_id = label[4]
            x = [label[0], label[1], label[2], label[3]]
            x = pascal_voc_to_yolo(x, transformed_image.shape[1], transformed_image.shape[0])
            f.write(f'{int(class_id)} {x[0]:.6f} {x[1]:.6f} {x[2]:.6f} {x[3]:.6f}\n')
        f.close()

C:/Users/User2/source/pcounter/data/people/train/images/p_train0000.png
C:/Users/User2/source/pcounter/data/people/train/labels/p_train0000.txt
C:/Users/User2/source/pcounter/data/people/train_aug/images/rot90VF_p_train0000.png
C:/Users/User2/source/pcounter/data/people/train_aug/labels/rot90VF_p_train0000.txt
3 0
[[307, 65, 373, 154, '0'], [440, 38, 488, 104, '0'], [275, 339, 389, 358, '0']]
[]
[(206.0, 267.0, 295.0, 333.0, '0'), (256.0, 152.00000000000003, 322.0, 200.0, '0'), (1.999999999999993, 251.0, 21.000000000000007, 365.0, '0')]
C:/Users/User2/source/pcounter/data/people/train/images/p_train0001.png
C:/Users/User2/source/pcounter/data/people/train/labels/p_train0001.txt
C:/Users/User2/source/pcounter/data/people/train_aug/images/rot90VF_p_train0001.png
C:/Users/User2/source/pcounter/data/people/train_aug/labels/rot90VF_p_train0001.txt
5 0
[[1, 23, 88, 92, '0'], [177, 84, 277, 158, '0'], [384, 83, 488, 158, '0'], [401, 1, 506, 106, '0'], [283, 335, 399, 359, '0']]
[]
[(1.0, 268.

In [18]:
# - 90도 랜덤 회전
# - 랜덤 회전 
# - 90도 랜덤 회전 + 수평 플립  
# - 랜덤 회전 + 수평 플립  
# - 90도 랜덤 회전 + 수직 플립
# ==> 랜덤 회전 + 수직 플립     
# - random crop 적용

# 추가 수정 사항 : 수평 수직 플립 적용시 rot를 하게 됨에 따른 재라벨링 작업을 없애기 위함.
#                 rot + 재라벨링 결과를 가지고 수평 수직 플립만 추가하도록 하여 재라벨링 작업 제거
# 수정이유 : rot하면 rot변형 라벨링 과정에서 bbox가 커지기 때문에 다시 수동으로 재라벨링해야 함
# 수정방향 : rot한 + 재라벨링 한 결과에 H/V flip만 하여 재라벨링 작업 제거 하려고 함.
# 준비사항 : train_aug폴더 아래에 train_rot 폴더를 만들고 이전에 rot한 뒤 라벨링한 images와 labels를 동일한 폴더 구조로 카피한다.
#  

SOURCE_PATH = 'C:/Users/User2/source/pcounter/data/people/train_aug/train_rot'
SOURCE_DATA_PATH = f'{SOURCE_PATH}'+'/images'
SOURCE_LABEL_PATH = f'{SOURCE_PATH}'+'/labels'

TARGET_PATH = 'C:/Users/User2/source/pcounter/data/people/train_aug'
TARGET_DATA_PATH = f'{TARGET_PATH}'+'/images'
TARGET_LABEL_PATH = f'{TARGET_PATH}'+'/labels'

image_files = os.listdir(SOURCE_DATA_PATH)

augment_type = 'rotVF_'
for filename in image_files:    
    pat, ext = os.path.splitext(filename)
    src_image_file = f'{SOURCE_DATA_PATH}/'+filename
    src_label_file = f'{SOURCE_LABEL_PATH}/'+pat+'.txt'        
    print(src_image_file)
    print(src_label_file)

    dst_image_file = f'{TARGET_DATA_PATH}/'+augment_type+filename     
    dst_label_file = f'{TARGET_LABEL_PATH}/'+augment_type+pat+'.txt'  
    print(dst_image_file)
    print(dst_label_file)

    bboxes = []
    category_ids = []

    image = cv2.imread(src_image_file, cv2.IMREAD_COLOR)    
    print(f'image size : {image.shape[1]}, {image.shape[0]}')
    
    if os.path.isfile(src_label_file):
        f = open(src_label_file, 'r')
        ##### yolo -> coco 변환 + make bboxes
        for label in f.readlines():
            x = label.split()
            class_id = x.pop(0)

            x = yolo_to_coco([float(x[0]), float(x[1]), float(x[2]), float(x[3])], image.shape[1], image.shape[0])
            x.append(str(class_id))
            bboxes.append(x)
        f.close()

    print(len(bboxes), len(category_ids))
    print(bboxes)
    print(category_ids)

    random.seed()    
    transform = A.Compose(
        [
            # A.Rotate(border_mode=cv2.BORDER_CONSTANT, value=[0,0,0], p=1),
            # A.RandomRotate90(p=1),
            # A.HorizontalFlip(p=1),
            A.VerticalFlip(p=1)
        ],
        bbox_params=A.BboxParams(format='coco'), #현재 albumentations의 버그 때문에 coco 형태로 변형함 => yolo_to_coco(), coco_to_yolo() 함수에서 오류가 있어 수정함
    ) 
    transformed = transform(image=image, bboxes=bboxes)
    transformed_image = transformed['image']
    transformed_bboxes = transformed['bboxes']

    cv2.imwrite(dst_image_file, transformed_image)
    print(transformed_bboxes)

    if len(bboxes) > 0:
        f = open(dst_label_file, 'w')
        for label in transformed_bboxes:
            class_id = label[4]
            x = [label[0], label[1], label[2], label[3]]
            # x = pascal_voc_to_yolo(x, transformed_image.shape[1], transformed_image.shape[0])
            x = coco_to_yolo(x, transformed_image.shape[1], transformed_image.shape[0])
            f.write(f'{int(class_id)} {x[0]:.6f} {x[1]:.6f} {x[2]:.6f} {x[3]:.6f}\n')
        f.close()

C:/Users/User2/source/pcounter/data/people/train_aug/train_rot/images/rot_p_train0000.png
C:/Users/User2/source/pcounter/data/people/train_aug/train_rot/labels/rot_p_train0000.txt
C:/Users/User2/source/pcounter/data/people/train_aug/images/rotVF_rot_p_train0000.png
C:/Users/User2/source/pcounter/data/people/train_aug/labels/rotVF_rot_p_train0000.txt
image size : 640, 360
3 0
[[331, 113, 96, 65, '0'], [459, 191, 85, 48, '0'], [165, 265, 96, 91, '0']]
[]
[(331.0, 182.0, 96.0, 65.0, '0'), (459.0, 121.00000000000001, 85.0, 47.999999999999986, '0'), (165.0, 3.999999999999986, 96.0, 91.0, '0')]
C:/Users/User2/source/pcounter/data/people/train_aug/train_rot/images/rot_p_train0001.png
C:/Users/User2/source/pcounter/data/people/train_aug/train_rot/labels/rot_p_train0001.txt
C:/Users/User2/source/pcounter/data/people/train_aug/images/rotVF_rot_p_train0001.png
C:/Users/User2/source/pcounter/data/people/train_aug/labels/rotVF_rot_p_train0001.txt
image size : 640, 360
5 0
[[1, 246, 80, 67, '0'], [1

In [9]:
# - 90도 랜덤 회전
# - 랜덤 회전 
# - 90도 랜덤 회전 + 수평 플립  
# - 랜덤 회전 + 수평 플립  
# - 90도 랜덤 회전 + 수직 플립
# - 랜덤 회전 + 수직 플립     
# ==> cut out 적용
augment_type = 'cut_'
for filename in image_files:    
    pat, ext = os.path.splitext(filename)
    src_image_file = f'{SOURCE_DATA_PATH}/'+filename
    src_label_file = f'{SOURCE_LABEL_PATH}/'+pat+'.txt'        
    print(src_image_file)
    print(src_label_file)

    dst_image_file = f'{TARGET_DATA_PATH}/'+augment_type+filename     
    dst_label_file = f'{TARGET_LABEL_PATH}/'+augment_type+pat+'.txt'  
    print(dst_image_file)
    print(dst_label_file)

    bboxes = []
    category_ids = []

    image = cv2.imread(src_image_file, cv2.IMREAD_COLOR)    

    if os.path.isfile(src_label_file):
        f = open(src_label_file, 'r')
        for label in f.readlines():
            x = label.split()
            class_id = x.pop(0)

            x = yolo_to_pascal_voc([float(x[0]), float(x[1]), float(x[2]), float(x[3])], image.shape[1], image.shape[0])
            x.append(str(class_id))
            bboxes.append(x)
        f.close()

    print(len(bboxes), len(category_ids))
    print(bboxes)
    print(category_ids)

    random.seed()    
    transform = A.Compose(
        [
            # A.Rotate(border_mode=cv2.BORDER_CONSTANT, value=[0,0,0], p=1),
            # A.RandomRotate90(p=1),
            # A.HorizontalFlip(p=1),
            # A.VerticalFlip(p=1)
            A.Cutout(num_holes=3, max_h_size=30, max_w_size=30, p=1)
        ],
        bbox_params=A.BboxParams(format='pascal_voc'), #현재 albumentations의 버그 때문에 pascal_voc 형태로 변형하고 마지막에 다시 yolo 형태로 변형함
    ) 
    transformed = transform(image=image, bboxes=bboxes)
    transformed_image = transformed['image']
    transformed_bboxes = transformed['bboxes']

    cv2.imwrite(dst_image_file, transformed_image)
    print(transformed_bboxes)

    if len(bboxes) > 0:
        f = open(dst_label_file, 'w')
        for label in transformed_bboxes:
            class_id = label[4]
            x = [label[0], label[1], label[2], label[3]]
            x = pascal_voc_to_yolo(x, transformed_image.shape[1], transformed_image.shape[0])
            f.write(f'{int(class_id)} {x[0]:.6f} {x[1]:.6f} {x[2]:.6f} {x[3]:.6f}\n')
        f.close()

C:/Users/User2/source/pcounter/data/people/train/images/p_train0000.png
C:/Users/User2/source/pcounter/data/people/train/labels/p_train0000.txt
C:/Users/User2/source/pcounter/data/people/train_aug/images/cut_p_train0000.png
C:/Users/User2/source/pcounter/data/people/train_aug/labels/cut_p_train0000.txt
3 0
[[307, 65, 373, 154, '0'], [440, 38, 488, 104, '0'], [275, 339, 389, 358, '0']]
[]
[(307.0, 65.0, 373.0, 154.0, '0'), (440.0, 38.0, 488.0, 103.99999999999999, '0'), (275.0, 339.0, 389.0, 358.0, '0')]
C:/Users/User2/source/pcounter/data/people/train/images/p_train0001.png
C:/Users/User2/source/pcounter/data/people/train/labels/p_train0001.txt
C:/Users/User2/source/pcounter/data/people/train_aug/images/cut_p_train0001.png
C:/Users/User2/source/pcounter/data/people/train_aug/labels/cut_p_train0001.txt
5 0
[[1, 23, 88, 92, '0'], [177, 84, 277, 158, '0'], [384, 83, 488, 158, '0'], [401, 1, 506, 106, '0'], [283, 335, 399, 359, '0']]
[]
[(1.0, 23.0, 88.0, 92.0, '0'), (177.0, 84.0, 277.0, 15

c:\Users\User2\anaconda3\envs\btrack2\lib\site-packages\albumentations\augmentations\dropout\cutout.py:49: FutureWarning: Cutout has been deprecated. Please use CoarseDropout
  warnings.warn(


[(236.0, 22.0, 316.0, 108.0, '0'), (450.0, 27.0, 498.0, 101.0, '0'), (282.0, 338.0, 398.0, 360.0, '0')]
C:/Users/User2/source/pcounter/data/people/train/images/p_train0011.png
C:/Users/User2/source/pcounter/data/people/train/labels/p_train0011.txt
C:/Users/User2/source/pcounter/data/people/train_aug/images/cut_p_train0011.png
C:/Users/User2/source/pcounter/data/people/train_aug/labels/cut_p_train0011.txt
1 0
[[478, 68, 522, 102, '0']]
[]
[(478.0, 68.0, 522.0, 102.0, '0')]
C:/Users/User2/source/pcounter/data/people/train/images/p_train0012.png
C:/Users/User2/source/pcounter/data/people/train/labels/p_train0012.txt
C:/Users/User2/source/pcounter/data/people/train_aug/images/cut_p_train0012.png
C:/Users/User2/source/pcounter/data/people/train_aug/labels/cut_p_train0012.txt
7 0
[[1, 122, 138, 196, '0'], [223, 1, 273, 40, '0'], [1, 34, 156, 120, '0'], [1, 77, 39, 110, '0'], [1, 199, 50, 231, '0'], [292, 335, 407, 359, '0'], [465, 52, 506, 105, '0']]
[]
[(1.0, 122.0, 138.0, 195.9999999999999

In [10]:
# - 90도 랜덤 회전
# - 랜덤 회전 
# - 90도 랜덤 회전 + 수평 플립  
# - 랜덤 회전 + 수평 플립  
# - 90도 랜덤 회전 + 수직 플립
# - 랜덤 회전 + 수직 플립     
# ==> cut out 적용 10개
augment_type = 'cut10_'
for filename in image_files:    
    pat, ext = os.path.splitext(filename)
    src_image_file = f'{SOURCE_DATA_PATH}/'+filename
    src_label_file = f'{SOURCE_LABEL_PATH}/'+pat+'.txt'        
    print(src_image_file)
    print(src_label_file)

    dst_image_file = f'{TARGET_DATA_PATH}/'+augment_type+filename     
    dst_label_file = f'{TARGET_LABEL_PATH}/'+augment_type+pat+'.txt'  
    print(dst_image_file)
    print(dst_label_file)

    bboxes = []
    category_ids = []

    image = cv2.imread(src_image_file, cv2.IMREAD_COLOR)    

    if os.path.isfile(src_label_file):
        f = open(src_label_file, 'r')
        for label in f.readlines():
            x = label.split()
            class_id = x.pop(0)

            x = yolo_to_pascal_voc([float(x[0]), float(x[1]), float(x[2]), float(x[3])], image.shape[1], image.shape[0])
            x.append(str(class_id))
            bboxes.append(x)
        f.close()

    print(len(bboxes), len(category_ids))
    print(bboxes)
    print(category_ids)

    random.seed()    
    transform = A.Compose(
        [
            # A.Rotate(border_mode=cv2.BORDER_CONSTANT, value=[0,0,0], p=1),
            # A.RandomRotate90(p=1),
            # A.HorizontalFlip(p=1),
            # A.VerticalFlip(p=1)
            A.Cutout(num_holes=10, max_h_size=30, max_w_size=30, p=1)
        ],
        bbox_params=A.BboxParams(format='pascal_voc'), #현재 albumentations의 버그 때문에 pascal_voc 형태로 변형하고 마지막에 다시 yolo 형태로 변형함
    ) 
    transformed = transform(image=image, bboxes=bboxes)
    transformed_image = transformed['image']
    transformed_bboxes = transformed['bboxes']

    cv2.imwrite(dst_image_file, transformed_image)
    print(transformed_bboxes)

    if len(bboxes) > 0:
        f = open(dst_label_file, 'w')
        for label in transformed_bboxes:
            class_id = label[4]
            x = [label[0], label[1], label[2], label[3]]
            x = pascal_voc_to_yolo(x, transformed_image.shape[1], transformed_image.shape[0])
            f.write(f'{int(class_id)} {x[0]:.6f} {x[1]:.6f} {x[2]:.6f} {x[3]:.6f}\n')
        f.close()

C:/Users/User2/source/pcounter/data/people/train/images/p_train0000.png
C:/Users/User2/source/pcounter/data/people/train/labels/p_train0000.txt
C:/Users/User2/source/pcounter/data/people/train_aug/images/cut10_p_train0000.png
C:/Users/User2/source/pcounter/data/people/train_aug/labels/cut10_p_train0000.txt
3 0
[[307, 65, 373, 154, '0'], [440, 38, 488, 104, '0'], [275, 339, 389, 358, '0']]
[]
[(307.0, 65.0, 373.0, 154.0, '0'), (440.0, 38.0, 488.0, 103.99999999999999, '0'), (275.0, 339.0, 389.0, 358.0, '0')]
C:/Users/User2/source/pcounter/data/people/train/images/p_train0001.png
C:/Users/User2/source/pcounter/data/people/train/labels/p_train0001.txt
C:/Users/User2/source/pcounter/data/people/train_aug/images/cut10_p_train0001.png
C:/Users/User2/source/pcounter/data/people/train_aug/labels/cut10_p_train0001.txt
5 0
[[1, 23, 88, 92, '0'], [177, 84, 277, 158, '0'], [384, 83, 488, 158, '0'], [401, 1, 506, 106, '0'], [283, 335, 399, 359, '0']]
[]
[(1.0, 23.0, 88.0, 92.0, '0'), (177.0, 84.0, 2

In [11]:
# - 90도 랜덤 회전
# - 랜덤 회전 
# - 90도 랜덤 회전 + 수평 플립  
# - 랜덤 회전 + 수평 플립  
# - 90도 랜덤 회전 + 수직 플립
# - 랜덤 회전 + 수직 플립     
# ==> cut out 적용 15개 + 90도 랜덤 회전
augment_type = 'cut15rot90_'
for filename in image_files:    
    pat, ext = os.path.splitext(filename)
    src_image_file = f'{SOURCE_DATA_PATH}/'+filename
    src_label_file = f'{SOURCE_LABEL_PATH}/'+pat+'.txt'        
    print(src_image_file)
    print(src_label_file)

    dst_image_file = f'{TARGET_DATA_PATH}/'+augment_type+filename     
    dst_label_file = f'{TARGET_LABEL_PATH}/'+augment_type+pat+'.txt'  
    print(dst_image_file)
    print(dst_label_file)

    bboxes = []
    category_ids = []

    image = cv2.imread(src_image_file, cv2.IMREAD_COLOR)    

    if os.path.isfile(src_label_file):
        f = open(src_label_file, 'r')
        for label in f.readlines():
            x = label.split()
            class_id = x.pop(0)

            x = yolo_to_pascal_voc([float(x[0]), float(x[1]), float(x[2]), float(x[3])], image.shape[1], image.shape[0])
            x.append(str(class_id))
            bboxes.append(x)
        f.close()

    print(len(bboxes), len(category_ids))
    print(bboxes)
    print(category_ids)

    random.seed()    
    transform = A.Compose(
        [
            # A.Rotate(border_mode=cv2.BORDER_CONSTANT, value=[0,0,0], p=1),
            A.RandomRotate90(p=1),
            # A.HorizontalFlip(p=1),
            # A.VerticalFlip(p=1)
            A.Cutout(num_holes=10, max_h_size=30, max_w_size=30, p=1)
        ],
        bbox_params=A.BboxParams(format='pascal_voc'), #현재 albumentations의 버그 때문에 pascal_voc 형태로 변형하고 마지막에 다시 yolo 형태로 변형함
    ) 
    transformed = transform(image=image, bboxes=bboxes)
    transformed_image = transformed['image']
    transformed_bboxes = transformed['bboxes']

    cv2.imwrite(dst_image_file, transformed_image)
    print(transformed_bboxes)

    if len(bboxes) > 0:
        f = open(dst_label_file, 'w')
        for label in transformed_bboxes:
            class_id = label[4]
            x = [label[0], label[1], label[2], label[3]]
            x = pascal_voc_to_yolo(x, transformed_image.shape[1], transformed_image.shape[0])
            f.write(f'{int(class_id)} {x[0]:.6f} {x[1]:.6f} {x[2]:.6f} {x[3]:.6f}\n')
        f.close()

C:/Users/User2/source/pcounter/data/people/train/images/p_train0000.png
C:/Users/User2/source/pcounter/data/people/train/labels/p_train0000.txt
C:/Users/User2/source/pcounter/data/people/train_aug/images/cut15rot90_p_train0000.png
C:/Users/User2/source/pcounter/data/people/train_aug/labels/cut15rot90_p_train0000.txt
3 0
[[307, 65, 373, 154, '0'], [440, 38, 488, 104, '0'], [275, 339, 389, 358, '0']]
[]
[(307.0, 65.0, 373.0, 154.0, '0'), (440.0, 38.0, 488.0, 103.99999999999999, '0'), (275.0, 339.0, 389.0, 358.0, '0')]
C:/Users/User2/source/pcounter/data/people/train/images/p_train0001.png
C:/Users/User2/source/pcounter/data/people/train/labels/p_train0001.txt
C:/Users/User2/source/pcounter/data/people/train_aug/images/cut15rot90_p_train0001.png
C:/Users/User2/source/pcounter/data/people/train_aug/labels/cut15rot90_p_train0001.txt
5 0
[[1, 23, 88, 92, '0'], [177, 84, 277, 158, '0'], [384, 83, 488, 158, '0'], [401, 1, 506, 106, '0'], [283, 335, 399, 359, '0']]
[]
[(268.0, 1.0, 337.0, 88.0,